In [4]:
!pip install mesa_geo
import mesa_geo as mg
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import mesa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
nyB = gpd.GeoDataFrame.from_file("new-york-city-boroughs.geojson")

ERROR:fiona._env:new-york-city-boroughs.geojson: No such file or directory


DriverError: ignored

In [6]:
print(nyB.head())
nyB.plot()

NameError: ignored

In [7]:
man_gpd = nyB[nyB['name'] == 'Manhattan']
man_geo = man_gpd.geometry.iloc[0]

NameError: ignored

In [ ]:
bounds = man_gpd.total_bounds

In [ ]:
#rasterizing manhattan geojson file
#originally was to use as bounds for generation of model and incorporate somethign with cells but didn't have time to get it working
from rasterio.features import rasterize
res = 0.00001
transform = rasterio.transform.from_origin(bounds[0], bounds[3], res, res)
width = int((bounds[2] - bounds[0]) / res)
height = int((bounds[3] - bounds[1]) / res)
#rasterize manhattan multipolygon
transform = rasterio.transform.from_origin(bounds[0], bounds[3], res, res)
with rasterio.open('nyc_boroughs.tif', 'w', driver='GTiff', height=height, width=width, count=1, dtype='uint8', crs=nyB.crs, transform=transform) as out_file:
    out_file.write(rasterize(man_gpd.geometry, out_shape=(height, width), transform=transform, dtype='uint8'), 1)


In [ ]:
with rasterio.open('nyc_boroughs.tif') as src:
    raster_data = src.read(1)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

ax.imshow(raster_data, cmap='gray')

plt.show()

In [ ]:
man_gpd.total_bounds

In [ ]:
import mesa
import mesa_geo as mg
from shapely.geometry import Point

#for data collector
def infected_count(model):
    return (sum(1 for agent in model.schedule.agents if agent.atype == "infected"))

def susceptible_count(model):
    return (sum(1 for agent in model.schedule.agents if agent.atype == "susceptible"))

def dead_count(model):
    return (sum(1 for agent in model.schedule.agents if agent.atype == "dead"))

class Person(mg.GeoAgent):
    def __init__(self, unique_id, model,geometry,crs,agent_type="susceptible",range=150,recovery_rate=0.2,death_risk=0.1,init_infected=0.1,):
        super().__init__(unique_id, model, geometry, crs)
        self.atype = agent_type
        self.range = range
        self.recovery_rate = recovery_rate
        self.death_risk = death_risk
        #chance of person starting as infected
        if self.random.random() < init_infected:
            self.atype = "infected"

    def move_point(self, dx, dy):
        return Point(self.geometry.x + dx, self.geometry.y + dy)

    def step(self):
        self.check_status()
        self.move()

    def check_status(self):
        #infects
        if self.atype == "infected":
            if self.random.random() < self.recovery_rate:
                self.atype = "susceptible"
            elif self.random.random() < self.death_risk:
                self.atype = "dead"
        if self.atype == "infected":
            neighbors = self.model.space.get_neighbors_within_distance(self, self.model.exposure_distance)
            for neighbor in neighbors:
                if (neighbor.atype == "susceptible" and self.random.random() < self.model.infection_risk):
                    neighbor.atype = "infected"
                    break

    def move(self):

        if self.atype != "dead":
            move_x = self.random.randint(-self.range, self.range)
            move_y = self.random.randint(-self.range, self.range)
            self.geometry = Point(self.geometry.x + move_x, self.geometry.y + move_y)
        '''
        if self.atype != "dead":
            while True:
                move_x = self.random.randint(-self.range, self.range)
                move_y = self.random.randint(-self.range, self.range)
                newLoc = Point(self.geometry.x + move_x, self.geometry.y + move_y)
                neighbors = [agent for agent in self.model.space.get_intersecting_agents(newLoc) if isinstance(agent, BoroughAgent)]
                if neighbors:
                    self.geometry = newLoc
                    break
        '''
    def __repr__(self):
        return "Person " + str(self.unique_id)


class BoroughAgent(mg.GeoAgent):
    def __init__(self, unique_id, model, geometry, crs, agent_type="safe", hotspot_threshold=5):
        super().__init__(unique_id, model, geometry, crs)
        self.atype = agent_type
        self.hotspot_threshold = (
            hotspot_threshold
        )
        self.color_hotspot()

    def step(self):
        self.color_hotspot()

    def color_hotspot(self):
        # Region is hotspot if more agents infected within than above threshold
        neighbors = self.model.space.get_intersecting_agents(self)
        infected_neighbors = [
            neighbor for neighbor in neighbors if neighbor.atype == "infected"
        ]
        if len(infected_neighbors) <= self.hotspot_threshold:
            self.atype = "safe"
        else:
            self.atype = "hotspot"

    def __repr__(self):
        return "Neighborhood " + str(self.unique_id)




In [ ]:
class GeoInf(mesa.Model):
    # Geographical parameters for desired map
    geojson_regions = "new-york-city-boroughs.geojson"
    unique_id = "name"

    def __init__(
        self, n=30, percInf=0.2, expDist=500, infRisk=0.2
    ):
        self.schedule = mesa.time.BaseScheduler(self)
        self.space = mg.GeoSpace(warn_crs_conversion = False)

        # SIR model parameters
        self.pop = n
        self.exposure_distance = expDist
        self.infection_risk = infRisk

        self.running = True
        self.datacollector = mesa.DataCollector(
            {
                "infected": infected_count,
                "susceptible": susceptible_count,
                "dead": dead_count,
            }
        )
      # Create Borough agents
        ac = mg.AgentCreator(BoroughAgent, model=self)
        borough_agents = ac.from_file(
            self.geojson_regions, unique_id=self.unique_id
        )
        self.space.add_agents(borough_agents)

        # Create persons
        ac_population = mg.AgentCreator(
            Person,
            model=self,
            crs=self.space.crs,
            agent_kwargs={"init_infected": percInf},
        )
        # Add person to random borough in random location
        for i in range(n):
            #keep trying until agent is within a borough
            while True:
                borough = self.random.randint(0, len(borough_agents) - 1)
                center_x, center_y = borough_agents[borough].geometry.centroid.coords.xy
                bounds = borough_agents[borough].geometry.bounds
                spread_x = int(bounds[2] - bounds[0])
                spread_y = int(bounds[3] - bounds[1])

                x = center_x[0] + self.random.randint(0, spread_x) - spread_x / 2
                y = center_y[0] + self.random.randint(0, spread_y) - spread_y / 2
                new_person = ac_population.create_agent(Point(x, y), "P" + str(i))

                if borough_agents[borough].geometry.intersects(new_person.geometry):
                    self.space.add_agents(new_person)
                    self.schedule.add(new_person)
                    break

        for agent in borough_agents:
            self.schedule.add(agent)
        self.datacollector.collect(self)


    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

        # Run until no one is infected
        if infected_count(self) == 0:
            self.running = False

In [ ]:
model_params = {
    "n": mesa.visualization.Slider("Population size", 100, 10, 200, 10),
    "percInf": mesa.visualization.Slider(
        "Percent infected", 0.5, 0.00, 1.0, 0.05),
    "expDist": mesa.visualization.Slider(
        "Exposure distance", 600, 100, 1000, 100),
}


def portrayAgents(agent):
    portrayal = {}
    if isinstance(agent, Person):
        portrayal["radius"] = "2"
    if agent.atype in ["hotspot", "infected"]:
        portrayal["color"] = "Red"
    elif agent.atype in ["safe", "susceptible"]:
        portrayal["color"] = "Green"
    elif agent.atype in ["dead"]:
        portrayal["color"] = "Black"
    return portrayal

map = mg.visualization.MapModule(portrayAgents)
chart = mesa.visualization.ChartModule(
    [
        {"Label": "infected", "Color": "Red"},
        {"Label": "susceptible", "Color": "Green"},
        {"Label": "dead", "Color": "Black"},
    ]
)
server = mesa.visualization.ModularServer(
    GeoInf,
    [map, chart],
    "Basic Geo Infection model",
    model_params,
)


In [ ]:
server.launch()